# img transform

In [1]:
import pose

In [2]:
config=pose.__config__

In [3]:
import ref as ds

In [ ]:
ds.init()

loading data...


In [ ]:
orig_keypoint=ds.get_kps(0)
c=ds.get_center(0)
s=ds.get_scale(0)

In [ ]:
input_res=config['train']['input_res']

In [37]:
import numpy as np
def get_transform(center, scale, res, rot=0):
    # Generate transformation matrix
    h = 200 * scale
    t = np.zeros((3, 3))
    t[0, 0] = float(res[1]) / h
    t[1, 1] = float(res[0]) / h
    t[0, 2] = res[1] * (-float(center[0]) / h + .5)
    t[1, 2] = res[0] * (-float(center[1]) / h + .5)
    t[2, 2] = 1
    print("t : ")
    print(t)
    if not rot == 0:
        rot = -rot # To match direction of rotation from cropping
        rot_mat = np.zeros((3,3))
        rot_rad = rot * np.pi / 180
        sn,cs = np.sin(rot_rad), np.cos(rot_rad)
        rot_mat[0,:2] = [cs, -sn]
        rot_mat[1,:2] = [sn, cs]
        rot_mat[2,2] = 1
        # Need to rotate around center
        t_mat = np.eye(3)
        t_mat[0,2] = -res[1]/2
        t_mat[1,2] = -res[0]/2
        t_inv = t_mat.copy()
        t_inv[:2,2] *= -1
        t = np.dot(t_inv,np.dot(rot_mat,np.dot(t_mat,t)))
    return t

In [38]:
get_transform(c,s,(input_res,input_res))

t : 
[[  0.33895543   0.         -73.33952428]
 [  0.           0.33895543  25.52845498]
 [  0.           0.           1.        ]]


array([[  0.33895543,   0.        , -73.33952428],
       [  0.        ,   0.33895543,  25.52845498],
       [  0.        ,   0.        ,   1.        ]])

In [41]:
def transform(pt, center, scale, res, invert=0, rot=0):
    print("keypoint : ",pt)
    # Transform pixel location to different reference
    t = get_transform(center, scale, res, rot=rot)
    if invert:
        #normal => Not true
        t = np.linalg.inv(t)
    new_pt = np.array([pt[0], pt[1], 1.]).T
    print("new_keypoint : ", new_pt)
    new_pt = np.dot(t, new_pt)
    print("final_keypoint : ",new_pt)
    return new_pt[:2].astype(int)

In [42]:
transform(orig_keypoint[0,0,:2],c,s,(input_res,input_res))

keypoint :  [620. 394.]
t : 
[[  0.33895543   0.         -73.33952428]
 [  0.           0.33895543  25.52845498]
 [  0.           0.           1.        ]]
new_keypoint :  [620. 394.   1.]
final_keypoint :  [136.81284113 159.07689365   1.        ]


array([136, 159])

In [26]:
orig_keypoint[0,0,:2]

array([620., 394.])